In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
import awq

In [4]:
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from functools import partial
import gc

In [5]:
def evaluate(model, tokenizer):
    testenc = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    testenc = tokenizer("\n\n".join(testenc["text"]), return_tensors="pt")

    testenc = testenc.input_ids.to(model.device)
    nsamples = 40
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(
            shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
        )
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

In [6]:
def get_model_size(model: nn.Module, data_width=16, group_size=-1):

    if group_size != -1:
        data_width += (16 + 4) / group_size

    num_elements = 0
    for param in model.parameters():
        num_elements += param.numel()
    return num_elements * data_width


Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

In [7]:
model_path = "facebook/opt-1.3b"

# model_path = "facebook/opt-13b"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

/home/sizheli/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
original_model_n_bits = 32
torch_dtype = torch.float16 if original_model_n_bits == 16 else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

/home/sizheli/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
model_size = get_model_size(model, data_width=original_model_n_bits, group_size=128)

### Print the results
print(f"\nmodel perplexity: {model_perplexity:.2f}")
print(f"model size: {model_size/MiB:.2f} MiB")

evaluating...: 100%|██████████| 40/40 [00:20<00:00,  1.93it/s]



model perplexity: 14.47
model size: 5043.73 MiB


In [16]:
from awq.quantize.pre_quant_lester import run_awq, apply_awq
from typing import Literal

ActQuantType = Literal["per_token", "per_tensor", "none"]

q_config = {
    "zero_point": True,  # by default True
    "q_group_size": 128,  # whether to use group quantization
    "w_n_bits": 8,
    "a_n_bits": 8,
    "act_quant": "per_tensor",
}

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

awq_results = run_awq(
    model,
    tokenizer,
    w_bit=q_config["w_n_bits"],
    q_config=q_config,
    n_samples=128,
    seqlen=512,
)

Repo card metadata block was not found. Setting CardData to empty.


 * Split into 60 blocks


Running AWQ...:   0%|          | 0/24 [00:00<?, ?it/s]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 0.00427306117489934
ratio: 0.05, loss: 0.004263454116880894
ratio: 0.1, loss: 0.004256470128893852
ratio: 0.15, loss: 0.004252363927662373
ratio: 0.2, loss: 0.0042498731054365635
ratio: 0.25, loss: 0.004248196259140968
ratio: 0.3, loss: 0.004247210454195738
ratio: 0.35, loss: 0.004246462602168322
ratio: 0.4, loss: 0.004246068187057972
ratio: 0.45, loss: 0.0042457422241568565
ratio: 0.5, loss: 0.004245585296303034
ratio: 0.55, loss: 0.004245635122060776
ratio: 0.6, loss: 0.004245671909302473
ratio: 0.65, loss: 0.00424604257568717
ratio: 0.7, loss: 0.004246539901942015
ratio: 0.75, loss: 0.0042471387423574924
ratio: 0.8, loss: 0.004248109646141529
ratio: 0.85, loss: 0.004249550402164459
ratio: 0.9, loss: 0.004252143204212189
ratio: 0.95, loss: 0.0

Running AWQ...:   4%|▍         | 1/24 [00:06<02:39,  6.95s/it]

...

Running AWQ...:  92%|█████████▏| 22/24 [03:03<00:16,  8.47s/it]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 0.013264557346701622
ratio: 0.05, loss: 0.01326411310583353
ratio: 0.1, loss: 0.013263403438031673
ratio: 0.15, loss: 0.013262837193906307
ratio: 0.2, loss: 0.013263304717838764
ratio: 0.25, loss: 0.013262695632874966
ratio: 0.3, loss: 0.013261978514492512
ratio: 0.35, loss: 0.013262277469038963
ratio: 0.4, loss: 0.013262133114039898
ratio: 0.45, loss: 0.013261237181723118
ratio: 0.5, loss: 0.013261817395687103
ratio: 0.55, loss: 0.013261503539979458
ratio: 0.6, loss: 0.013261623680591583
ratio: 0.65, loss: 0.013261726126074791
ratio: 0.7, loss: 0.013262137770652771
ratio: 0.75, loss: 0.013262114487588406
ratio: 0.8, loss: 0.013262744061648846
ratio: 0.85, loss: 0.013262959197163582
ratio: 0.9, loss: 0.0132635198533535
ratio: 0.95, loss: 0.01326

Running AWQ...:  96%|█████████▌| 23/24 [03:12<00:08,  8.57s/it]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 0.0071968501433730125
ratio: 0.05, loss: 0.0071961767971515656
ratio: 0.1, loss: 0.007195735350251198
ratio: 0.15, loss: 0.007195128593593836
ratio: 0.2, loss: 0.007194618694484234
ratio: 0.25, loss: 0.007194302044808865
ratio: 0.3, loss: 0.007194048725068569
ratio: 0.35, loss: 0.007193746976554394
ratio: 0.4, loss: 0.007193713914602995
ratio: 0.45, loss: 0.007193513214588165
ratio: 0.5, loss: 0.007193679455667734
ratio: 0.55, loss: 0.007193423807621002
ratio: 0.6, loss: 0.007193758152425289
ratio: 0.65, loss: 0.007194096688181162
ratio: 0.7, loss: 0.00719437887892127
ratio: 0.75, loss: 0.007195138838142157
ratio: 0.8, loss: 0.00719570554792881
ratio: 0.85, loss: 0.007196441292762756
ratio: 0.9, loss: 0.007197371684014797
ratio: 0.95, loss: 0.00

Running AWQ...: 100%|██████████| 24/24 [03:20<00:00,  8.37s/it]


In [17]:
dump_awq = "awq_results.pt"
torch.save(awq_results, dump_awq)
print("AWQ results saved at", dump_awq)
#

AWQ results saved at awq_results.pt


In [ ]:
### load awq

load_awq = "awq_results.pt"
awq_results = torch.load(load_awq, map_location="cpu")

In [13]:
q_config

{'zero_point': True,
 'q_group_size': 128,
 'w_n_bits': 8,
 'a_n_bits': 8,
 'act_quant': 'per_tensor'}

In [18]:

from transformers.models.opt.modeling_opt import OPTDecoderLayer
from awq.quantize.fake_quant_lester import quantize_opt_model

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

# apply the AWQ results
apply_awq(model, awq_results)

model = quantize_opt_model(
    model,
    w_n_bits=q_config["w_n_bits"],
    a_n_bits=q_config["a_n_bits"],
    act_quant=q_config["act_quant"],
)


In [19]:
torch.cuda.empty_cache()
model.cuda()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): QuantizedLinear()
          (fc2): QuantizedLinear()
          (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
 

In [20]:
# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
model_size = get_model_size(model, data_width=q_config["w_n_bits"], group_size=128)
print(f"\nmodel perplexity: {model_perplexity:.2f}")
print(f"model size: {model_size/MiB:.2f} MiB")

evaluating...: 100%|██████████| 40/40 [00:22<00:00,  1.81it/s]


model perplexity: 13702.26
model size: 496.07 MiB


In [14]:
print("hi")

hi


## RANDOM STUFF

In [ ]:
from awq.quantize.pre_quant_lester import get_blocks

In [ ]:
for layer in get_blocks(model):
    # layer.fc1
    # layer.fc2
    # print("hi")
    break

In [ ]:
from copy import deepcopy

x = deepcopy(layer)

In [ ]:
x.fc1.weight.data[:] = 1.0
# x.fc1.weight.data

layer.fc1.weight.data

In [ ]:
isinstance(layer, OPTDecoderLayer)